In [1]:
from configuration import ConfigurationBuilder
from pricing import BlackScholesMerton, GeometricBrownianMotion

### Replication : Put Down & In

In [2]:
configuration = ConfigurationBuilder(
    kind='put',
    spot=100.0,
    sigma=0.30,
    maturity=365,
    risk_free_rate=0,
    dividend_yield=0,
)

In [3]:
barrier = 70
shift = 3

configuration.strike = barrier
option_1 = BlackScholesMerton(configuration)

configuration.strike = barrier - shift
option_2 = BlackScholesMerton(configuration)

In [4]:
q = (configuration.spot - barrier) / shift
q

10.0

In [5]:
pdi = (option_1 * -1) * (q + 1) + option_2 * q

In [6]:
(pdi.price(), pdi.delta(), pdi.gamma(), pdi.vega(), pdi.theta(), pdi.rho())

(-5.43202077982245,
 0.30547008306025203,
 -0.015535227460800653,
 -0.4660568238240195,
 0.019153020157151493,
 0.35979029085847636)

In [7]:
configuration.strike = configuration.spot
configuration.simulation = 40000
configuration.steps = 365
gbm_pricing = GeometricBrownianMotion(configuration)
gbm_pricing.run_simulation()

In [9]:
gbm_pricing.put_down_in(barrier=barrier)

5.700760272824762